In [2]:
import torch 
import torchvision
from torchvision import datasets, transforms

For this tutorail, we will use the CIFAR10 dataset <br>
<h3>We will do the following steps in order</h3> <br>
<li>Load and normalize the dataset using torchvision </li>
<li>Difine a CNN </li>
<li>Difine a loss function </li>
<li>Training the network on the training </li>
<li>Test the network on the test data </li> <br>
The output of torchvesion dataset are PILimages of [0, 1]. We transfrom them to tensors of normalized range [-1, 1]

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(), 
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)
batch_size = 4
trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform
)
trainloder = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True, num_workers = 2)

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = True, num_workers = 2 )

# difine the classes 

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 
           'frog', 'horse', 
           'ship', 'truck')

In [ ]:
import matplotlib.pyplot as plt 
import numpy as np 
# function to show an image 

def imshow(img): 
    img = img / 2 + 0.5
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.imshow()
    
# get some random training images 
dataiter = iter(trainloder)
images, labels = dataiter.next()
#show images 

imshow(torchvision.utils.make_grid(images))
# print label 

print(' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))


<h2> Define a Convolutuion Neural network </h2>

In [ ]:
import torch.nn as nn 
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super.__inint___()
        self.conv1 = nn.Conv1d(3, 6, 5)
        self.pool  = nn.MaxPool2d(2, 2)
        self.conv2 =  nn.Conv2d(6, 16, 5)
        self.fct1  = nn.Linear(16*5*5, 120)
        self.fct2  = nn.Linear(120, 84)
        self.fct3  = nn.Linear(84, 10)
        
    def forward(self, x): 
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions exepet batch
        x = F.relu(self.fct1(x))
        x = F.relu(self.fct2(x))
        x = self.fct3(x)
        return x
    
net = Net()

<h2> Define a loss funcation and optimzer <h1><br>


In [ ]:
import torch.optim as optimzer 
#  We will use a classification Cross-Entropy loss and SGD momentum
criterion = nn.CrossEntropyLoss()
optimzer = optimzer.SGD(net.parameters, lr = 0.001, momentum = 0.9)

In [ ]:
for epoch in range(2): 
    running_loss = 0.0
    
    for i, data in enumerate(trainloder, 0):
        print('strat training ...')
        # get the inputs and labels; data is a list of [inputs, labels]
        inputs, labels = data
        
        #zero the parameters gradients 
        optimzer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimzer.step()
        # print statistics 
        running_loss += loss.item()
        if i % 2000 == 1999:
            # print every 2000 minibatches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')

print('Finished training.')
                 


In [ ]:
# save the model 
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

<h2>Test the network on the test datast </h2>

We have trained the network for 2 pass over the training dataset. But we need to check if the network <br>
has learnt anything at all. <br>
<br>
We will check this by predicting the class label that the neural netowk outputs, and cheking it against<br>
the ground-truth. If the prediction is correct, we add the sample of the list of correct prediction.<br>

In [ ]:
dataiter = iter(testloader)

images, labels = dataiter.next()
#print images 
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join(f'{classes[labels[j]]:5s}' for j in range(4)))

In [ ]:
#  let’s load back in our saved model

net = Net()
net.load_state_dict(torch.load(PATH))

In [ ]:
outputs = net(images)